In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline, DataCollatorForLanguageModeling
from torch.utils.data import Dataset, DataLoader
import os
from pathlib import Path
import torch

2022-01-26 17:44:13.039005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [2]:
import sys
sys.path.append("../../src")
from dfs_transformer.utils.rdkit import isValid
from rdkit import Chem
import numpy as np

Using backend: pytorch


In [3]:

#any model weights from the link above will work here
model = AutoModelWithLMHead.from_pretrained("seyonec/PubChem10M_SMILES_BPE_50k")#"seyonec/ChemBERTA_PubChem1M_shard00_155k")
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_50k")#"seyonec/ChemBERTA_PubChem1M_shard00_155k")

fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

/home/chrisw/.cache/pypoetry/virtualenvs/graph-transformer-9jPERXQ--py3.8/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:783: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/515 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/321M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/161k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [4]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [5]:
class RawTextDataset(Dataset):
    """
    Custom Torch Dataset for tokenizing large (up to 100,000,000+ sequences) text corpuses,
    by not loading the entire dataset into cache and using lazy loading from disk (using huggingface's
    'NLP' library. See 'https://github.com/huggingface/nlp' for more details on the NLP package.
    Examples
    --------
    >>> from raw_text_dataset import RawTextDataset
    >>> dataset = RawTextDataset(tokenizer=tokenizer, file_path="shard_00_selfies.txt", block_size=512)
    Downloading: 100%
    1.52k/1.52k [00:03<00:00, 447B/s]
    Using custom data configuration default
    Downloading and preparing dataset text/default-f719ef2eb3ab586b (download: Unknown size, generated: Unknown size, post-processed: Unknown sizetotal: Unknown size) to /root/.cache/huggingface/datasets/text/default-f719ef2eb3ab586b/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b...
    Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-f719ef2eb3ab586b/0.0.0/3a79870d85f1982d6a2af884fde86a71c771747b4b161fd302d28ad22adf985b. Subsequent calls will reuse this data.
    Loaded Dataset
    Number of lines: 999988
    Block size: 512
    """

    def __init__(self, tokenizer, file_path: str, block_size: int):
        super().__init__()
        self.tokenizer = tokenizer
        self.file_path = file_path
        self.block_size = block_size

        self.dataset = Path("/mnt/ssd/datasets/pubchemvalid.txt").read_text(encoding="utf-8").splitlines()
        print("Loaded Dataset")
        self.len = len(self.dataset)
        print("Number of lines: " + str(self.len))
        print("Block size: " + str(self.block_size))

    def __len__(self):
        return self.len

    def preprocess(self, feature_dict):
        batch_encoding = self.tokenizer(
            feature_dict,
            add_special_tokens=True,
            truncation=True,
            max_length=self.block_size,
        )
        return torch.tensor(batch_encoding["input_ids"])

    def __getitem__(self, i):
        line = self.dataset[i]
        example = self.preprocess(line)
        return example

In [6]:
dataset = RawTextDataset(tokenizer=tokenizer, file_path='/mnt/ssd/datasets/pubchemvalid.txt', block_size=512)

Loaded Dataset
Number of lines: 9942
Block size: 512


In [7]:
dl = DataLoader(dataset, batch_size=16, collate_fn = data_collator)

In [8]:
model = model.to('cuda:0')

In [ ]:
import tqdm
smiles = []
for data in tqdm.tqdm(dl):
    mask = data['labels']!=-100
    pred = data['input_ids'].clone()
    pred[mask] = model(data['input_ids'].to('cuda:0')).logits.cpu().argmax(dim=2)[mask]
    smiles += tokenizer.batch_decode(pred, skip_special_tokens=True)

 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                            | 519/622 [01:27<00:16,  6.16it/s]

In [ ]:
valid = np.asarray([isValid(Chem.MolFromSmiles(sml)) for sml in smiles])
print(valid.sum()/len(valid))

In [ ]:
orig_smiles = Path("/mnt/ssd/datasets/pubchemvalid.txt").read_text(encoding="utf-8").splitlines()

In [ ]:
same = []
for sml, osml in tqdm.tqdm(zip(smiles, orig_smiles)):
    try:
        csml1 = Chem.MolToSmiles(Chem.MolFromSmiles(sml))
        csml2 = Chem.MolToSmiles(Chem.MolFromSmiles(osml))
        same += [csml1 == csml2]
    except:
        continue
print(np.asarray(same).sum()/len(same))